In [10]:
import     numpy     as    np ,     pandas     as     pd
from     matplotlib . pyplot     import     subplots 
from     sklearn . linear_model     import     \
(LinearRegression , 
LogisticRegression , 
Lasso )
from sklearn . preprocessing import StandardScaler
from     sklearn . model_selection     import     KFold
from     sklearn . pipeline     import     Pipeline
from     ISLP     import     load_data
from     ISLP . models     import     ModelSpec     as     MS
from     sklearn . model_selection     import    \
    (train_test_split,GridSearchCV)

import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset
from torchmetrics import (MeanAbsoluteError,R2Score)
from torchinfo import summary
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

In [11]:
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")

x_train = train[["AI_Interaction_Level", "Satisfaction_with_AI_Services"]]
x_test = test[["AI_Interaction_Level", "Satisfaction_with_AI_Services"]]
y_train =train["Customer_Churn"]
y_test = test["Customer_Churn"]

In [7]:
features = ["AI_Interaction_Level", "Satisfaction_with_AI_Services"]

# Selecting only the specified features for X
allvars= train[features]
model = MS(allvars,intercept=False)
X = model.fit_transform(train).to_numpy()
Y  = train["Customer_Churn"].to_numpy()

In [5]:
class DataModel(nn.Module):
    def __init__(self,input_size):
        super(DataModel,self).__init__()
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(nn.Linear(input_size,50),nn.ReLU(),nn.Dropout(0.4),nn.Linear(50,1))
    def forward(self,x):
        x = self.flatten
        return torch.flatten(self.sequential(x))

In [12]:
data_model = DataModel(train.shape[1])
summary (data_model ,
input_size = x_train .shape , 
col_names =['input_size',
'output_size', 
'num_params'])


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [9]:

import numpy as np

# Assuming x_train, x_test are your features and y_train, y_test are the labels (0 or 1 for churn)

# Preprocess the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert labels to categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train_scaled, y_train_cat, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
model.evaluate(x_test_scaled, y_test_cat)


Epoch 1/50


c:\Users\30243\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5387 - loss: 0.6939 - val_accuracy: 0.5960 - val_loss: 0.6494
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.5617 - loss: 0.6751 - val_accuracy: 0.5960 - val_loss: 0.6455
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - accuracy: 0.5782 - loss: 0.6642 - val_accuracy: 0.5970 - val_loss: 0.6460
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 633us/step - accuracy: 0.5778 - loss: 0.6615 - val_accuracy: 0.5910 - val_loss: 0.6460
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step - accuracy: 0.5755 - loss: 0.6586 - val_accuracy: 0.5990 - val_loss: 0.6446
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step - accuracy: 0.5795 - loss: 0.6579 - val_accuracy: 0.5930 - val_loss: 0.6459
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step - accuracy: 0.5660 - loss: 0.6639 - val_accuracy: 0.5930 - val_loss: 0.6439
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 633us/step - accuracy: 0.5846 - loss: 0.6567 - val_accuracy:

[0.6442793011665344, 0.5680000185966492]